## C-ShipGen: Sample Tailored Ship Hulls from a Tabular DDPM

##### Set Up Tasks: Don't alter Please #####

In [21]:
# import the fun
import sys
import os 
sys.path.append('./tools')
sys.path.append('./data')

import numpy as np
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import Guided_Cond_DDPM_Tools as GC_DDPM
import pickle

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

from HullParameterization import Hull_Parameterization as HP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


np.set_printoptions(suppress=True) # don't use scientific notation

In [2]:
# Load in the Data:

#Step 1: Load in the data
DesVec = np.load('./data/DesVec_82k.npy', allow_pickle=True)
print(DesVec.shape)

DesVec_neg = np.load('./data/Negative_DesVec_82k.npy', allow_pickle=True)
print(DesVec_neg.shape)


# Now lets clean up X

idx_BBFactors = [33,34,35,36,37]
idx_BB = 31

idx_SBFactors = [38,39,40,41,42,43,44]
idx_SB = 32

for i in range(0,len(DesVec)):
    
    DesVec[i,idx_BBFactors] = DesVec[i,idx_BB] * DesVec[i,idx_BBFactors] 
    DesVec[i,idx_SBFactors] = DesVec[i,idx_SB] * DesVec[i,idx_SBFactors]



Y = np.load('./data/GeometricMeasures.npy', allow_pickle=True)

LenRatios = np.load('./data/Length_Ratios.npy', allow_pickle=True)


X_LIMITS = np.load('./data/X_LIMITS.npy')

print(X_LIMITS.shape)

X_lower_lim = [X_LIMITS[:,0].tolist()]                   
X_upper_lim = [X_LIMITS[:,1].tolist()]



(82168, 45)
(82793, 44)
(44, 2)


In [3]:
#Set up Conditioning Vectors:
num_WL_Steps = 101

VolVec = np.log10(Y[:,1*num_WL_Steps:2*num_WL_Steps])
idx = np.where(np.isnan(VolVec))
print(idx)

VolVec[idx] = -6.0 #fix nan to dummy value

print(VolVec.shape)

DdVec = DesVec[:,4]
BOAVec = np.amax(LenRatios[:,1:3], axis=1)
print(BOAVec.shape) 






(array([77257, 77257, 77257, 77257], dtype=int64), array([1, 2, 3, 4], dtype=int64))
(82168, 101)
(82168,)


C:\Users\vgnau\AppData\Local\Temp\ipykernel_26836\214063027.py:4: RuntimeWarning: invalid value encountered in log10
  VolVec = np.log10(Y[:,1*num_WL_Steps:2*num_WL_Steps])


In [4]:
# Set up the file for architecting the network, diffusion parameters, and training

DDPM_Dict = {
        'xdim' : len(DesVec[0])-1,             # Dimension of parametric design vector
        'datalength': len(DesVec),           # number of samples
        'X_LL' : X_lower_lim,           # lower limits of parametric design vector variables
        'X_UL' : X_upper_lim,
        'ydim': 0,                       # Number of objectives
        'cdim': 4,                      # number of conditioning inputs
        'gamma' : 0.2,                  # weight of feasibility guidance for guided sampling
        'lambda': [0.3,0.3],                 # weight of drag  guidance for guided sampling
        #'lambdas': [1,1,1,1,1,1,1],     # dummy variable for performance guided sampling
        'tdim': 128,                    # dimension of latent variable
        'net': [1024,1024,1024,1024],   # network architecture
        'batch_size': 1024,             # batch size
        'Training_Epochs': 10000,      # number of training epochs
        'Diffusion_Timesteps': 1000,    # number of diffusion timesteps
        'lr' : 0.00025,                 # learning rate
        'weight_decay': 0.0,            # weight decay
        'device_name': device}        # gpu device name


Classify_Dict = {
        'xdim' : len(DesVec[0])-1,
        'cdim': 1,
        'tdim': 128,
        'net': [64,64,64],
        'Training_Epochs': 150000,
        'device_name': device}

nodes = 512
Drag_Reg_Dict = {
        'xdim' : len(DesVec[0])-1,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 50000,  #30000             # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_CT',         # labels for regressors       
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': device} 

nodes = 256
LOA_wBulb_Reg_Dict = {
        'xdim' : len(DesVec[0])-1,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 150000,               # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_LOA_wBulb',         # labels for regressors
                    
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': device}   

WL_Reg_Dict = {
        "xdim": len(DesVec[0]),
        "ydim": 1, 
        "tdim": 512, 
        "net": [512, 512, 512], 
        "Training_Epochs": 30000, 
        "batch_size": 1024, 
        "Model_Label": 
        "Regressor_WL", 
        "lr": 0.001, 
        "weight_decay": 0.0, 
        "device_name": device}

Vol_Reg_Dict = {
                "xdim": len(DesVec[0]), 
                "ydim": 1, 
                "tdim": 512, 
                "net": [512, 512, 512], 
                "Training_Epochs": 30000, 
                "batch_size": 1024, 
                "Model_Label": "Regressor_WL", 
                "lr": 0.001, 
                "weight_decay": 0.0, 
                "device_name": device}




T = GC_DDPM.GuidedDiffusionEnv(DDPM_Dict,
                Classify_Dict,
                Drag_Reg_Dict,
                LOA_wBulb_Reg_Dict,
                WL_Reg_Dict,
                Vol_Reg_Dict,
                X= DesVec[:,1:],
                X_neg= DesVec_neg,
                VolVec = VolVec, 
                BOAVec = BOAVec, 
                DdVec = DdVec)




In [5]:
diffusion_path = './TrainedModels/CShipGen_diffusion.pth'
T.load_trained_diffusion_model(diffusion_path)

classifier_path = './TrainedModels/Constraint_Classifier_150000Epochs.pth' 

T.load_trained_classifier_model(classifier_path)


PATHS = ['./TrainedModels/Regressor_CT.pth',
        './TrainedModels/Regressor_LOA_wBulb.pth',
        './TrainedModels/Regressor_WL.pth',
        './TrainedModels/Regressor_Vol.pth']


T.load_trained_Drag_regression_models(PATHS)


c:\Users\vgnau\Documents\ArcturusHullOptimization\C_ShipGen-main\tools\Guided_Cond_DDPM_Tools.py:1194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.diffusion.load_stat

## Define the Ship's Principal Characteristics ##

In [15]:
#Sample from the Model:
num_samples = 100

Ship = np.array([1.22, .25, .127, .254 , .019,1.4]) #[LOA(m), Beam(m), Draft(m), Depth(m), Volume(m^3), U(m/s)] # This is for an aircraft carrier dimensioned ship



## Generate the Hulls using C-ShipGen ##

In [24]:

# Run the Loop on the other samples:

print('Generating Hulls')

LOA = Ship[0] #in meters
BoL = Ship[1]/LOA #beam to length ratio
ToD = Ship[2]/Ship[3] #Draft to depth ratio
DoL = Ship[3]/LOA #Depth to length ratio
Vol = np.log10(Ship[4]/LOA**3) # to normalize Volume by LOA**3

U = Ship[5]  #  12.86 #m/s  = 25 knots

dim_d = np.array([[ToD, U, LOA]]) #Drag_conditioning is [ToD, U(m/s), LOA (m)]

drag_cond = np.repeat(dim_d, num_samples, axis=0) #reapeat 
print(drag_cond.shape)
dim_g = np.array([[ToD, BoL, DoL, Vol]])

geom_cond = np.repeat(dim_g, num_samples, axis=0) #reapeat 
print(geom_cond.shape)

X_gen, unnorm = T.gen_vol_drag_guided_samples(geom_cond, drag_cond)

print(X_gen.shape)


Rt_guidance = T.Predict_Drag(unnorm, drag_cond)
Drag_Guidance = np.mean(Rt_guidance)


print('Predicted Mean Drag of Guidance samples: ' + str(Drag_Guidance) + ' N')
print('Minimum Drag of Guidance samples: ' + str(np.amin(Rt_guidance)) + ' N')


    

    

Generating Hulls
(100, 3)
(100, 4)


100%|██████████| 32/32 [00:00<00:00, 253.97it/s]

(100, 45)
Predicted Mean Drag of Guidance samples: 11.134827 N
Minimum Drag of Guidance samples: 0.51811755 N


### Clean up the Vectors and Check Feasibility ###

In [19]:

x_samples = X_gen

#print(x_samples[0:3])
    
print('Checking Feasibility of Samples')

for i in range(0,len(x_samples)):
    
    x_samples[i,idx_BB] = (x_samples[i,idx_BB] + 0.5) // 1 #int rounds to 1 or 0
    x_samples[i,idx_SB] = (x_samples[i,idx_SB] + 0.5) // 1 #int rounds to 1 or 0
    
    
    x_samples[i,idx_BBFactors] = x_samples[i,idx_BB] * x_samples[i,idx_BBFactors] 
    x_samples[i,idx_SBFactors] = x_samples[i,idx_SB] * x_samples[i,idx_SBFactors]



#Check the constraint violations for the sampled designs
constraints = []
sum_violation = []
cons = []
valid_idx = []

for i in tqdm(range(0,len(x_samples))):
    hull = HP(x_samples[i])
    constraints.append(hull.input_Constraints())
    cons.append(constraints[i] > 0)
    if sum(cons[i]) == 0:
        valid_idx.append(i)
        #hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)
    sum_violation.append(sum(cons[i]))

'''
print(len(valid_idx))
sample_vol = []
sample_BOA = []
sample_Dd = []
sample_LOA = []
sample_LOA_wBulb = []
idx_to_remove = []

for i in tqdm(range(0,len(valid_idx))):
    hull = HP(x_samples[valid_idx[i]]) 
    #print(i)
    try:
        Z = hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)    
        sample_vol.append(HP.interp(hull.Volumes, Z, Ship[2])) #interpolate to the draft of the ship
        BOA = max(hull.Calc_Max_Beam_midship(), hull.Calc_Max_Beam_PC())
        sample_BOA.append(BOA)
        sample_Dd.append(hull.Dd)
        sample_LOA.append(hull.LOA)
        sample_LOA_wBulb.append(hull.Calc_LOA_wBulb())
    except:
        print('error at hull {}'.format(i))
        idx_to_remove.append(i)

        continue

#Remove the samples that failed to calculate volume properties
valid_idx = np.delete(valid_idx, idx_to_remove)
print(len(valid_idx))
'''


Checking Feasibility of Samples


100%|██████████| 100/100 [00:00<00:00, 7508.20it/s]


"\nprint(len(valid_idx))\nsample_vol = []\nsample_BOA = []\nsample_Dd = []\nsample_LOA = []\nsample_LOA_wBulb = []\nidx_to_remove = []\n\nfor i in tqdm(range(0,len(valid_idx))):\n    hull = HP(x_samples[valid_idx[i]]) \n    #print(i)\n    try:\n        Z = hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)    \n        sample_vol.append(HP.interp(hull.Volumes, Z, Ship[2])) #interpolate to the draft of the ship\n        BOA = max(hull.Calc_Max_Beam_midship(), hull.Calc_Max_Beam_PC())\n        sample_BOA.append(BOA)\n        sample_Dd.append(hull.Dd)\n        sample_LOA.append(hull.LOA)\n        sample_LOA_wBulb.append(hull.Calc_LOA_wBulb())\n    except:\n        print('error at hull {}'.format(i))\n        idx_to_remove.append(i)\n\n        continue\n\n#Remove the samples that failed to calculate volume properties\nvalid_idx = np.delete(valid_idx, idx_to_remove)\nprint(len(valid_idx))\n"

### Generate Stl of valid Hull Designs ###

Note: 

Not all generated samples are feasible since C-ShipGen is a statistical Model.

Similarly, C-ShipGen does not generate hull designs exactly to the dimensions specified by the user; however, these designs are close to the intended dimensions. 

In [22]:
path = "./DEMO_Hulls/"

label = 'Demo_Hull_'

isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)

# print the indices of the 10 hulls with the lowest drag
print('Designs with Minimum Drag: ')
idxs = np.argsort(Rt_guidance[valid_idx].flatten())
for i in range(10):
    print(label + str(valid_idx[idxs[i]]))
    #save parameters of designs with minimum drag
with open('Min_Drag_Hull_Parameters.pkl', 'wb') as f:
    pickle.dump(x_samples[idxs[0:10]], f)
    

for i in tqdm(range(0,10)):
    Hull = HP(x_samples[valid_idx[idxs[i]]])
    #make the .stl file of the hull:
    strpath =  path+label + '_' + str(valid_idx[idxs[i]])
    try:
        mesh = Hull.gen_stl(NUM_WL=47, PointsPerWL=151, bit_AddTransom = 1, bit_AddDeckLid = 1, bit_RefineBowAndStern = 1,namepath = strpath)
    except:
        print('Error at hull {}'.format(valid_idx[idxs[i]]))

idx_min_pred_drag = np.argmin(Rt_guidance[valid_idx])
print('Minimum Predicted Drag: ' + str(Rt_guidance[valid_idx][idx_min_pred_drag][0]) + ' N')
print('Design with Minimum Predicted Drag: ')
print(label + str(valid_idx[idx_min_pred_drag])) #Highlight design with minimum predicted drag






Designs with Minimum Drag: 
Demo_Hull_85
Demo_Hull_43
Demo_Hull_20
Demo_Hull_69
Demo_Hull_89
Demo_Hull_23
Demo_Hull_2
Demo_Hull_91
Demo_Hull_9
Demo_Hull_39


100%|██████████| 10/10 [00:00<00:00, 12.98it/s]

Minimum Predicted Drag: 7.5563607 N
Design with Minimum Predicted Drag: 
Demo_Hull_85


In [10]:
# Design a hull with multiple fidelity levels
'''
Labels = ['Low', 'Medium', 'High']
WL = [47, 111, 203]
PPW = [151, 301, 601]

idx = 87

for i in tqdm(range(len(Labels))):
    Hull = HP(x_samples[idx])
    #make the .stl file of the hull:
    strpath =  path+label + '_' + str(idx) + '_' + Labels[i]
    try:
        mesh = Hull.gen_stl(NUM_WL=WL[i], PointsPerWL=PPW[i], bit_AddTransom = 1, bit_AddDeckLid = 1, bit_RefineBowAndStern = 1,namepath = strpath)
    except:
        print('Error at hull {}'.format(valid_idx[idxs[i]]))
'''

"\nLabels = ['Low', 'Medium', 'High']\nWL = [47, 111, 203]\nPPW = [151, 301, 601]\n\nidx = 87\n\nfor i in tqdm(range(len(Labels))):\n    Hull = HP(x_samples[idx])\n    #make the .stl file of the hull:\n    strpath =  path+label + '_' + str(idx) + '_' + Labels[i]\n    try:\n        mesh = Hull.gen_stl(NUM_WL=WL[i], PointsPerWL=PPW[i], bit_AddTransom = 1, bit_AddDeckLid = 1, bit_RefineBowAndStern = 1,namepath = strpath)\n    except:\n        print('Error at hull {}'.format(valid_idx[idxs[i]]))\n"

In [11]:
# Compare to Test Hull:

design = [[333,	0.587059958,	0.409996119,	0.122972918,	0.085,	0.71417088,	0.211,	0.120262544,	10.70168725,	0.264897988,	0.961214759,	-0.27,	0.15,	0.01,	0.3,	2.172250534,	-2.220618947,	0,	0.1,	0.05,	0,	0,	0.104249968,	0.37043854,	0.006300723,	-2.478684046,	2.882864263,	3.320624286,	0.076330577,	0.48538725,	0.455186006,	1,	1,	0.01,	0.4,	0.17155627,	0.38035235,	0.269331342,	0.7,	0.01,	1,	0.7,	0.025,	0.99,	0.147764723]]

print(len(design[0]))

hull = HP(design[0])

strpath =  path+'Test_Hull_Nimitz'
mesh = hull.gen_stl(NUM_WL=47, PointsPerWL=151, bit_AddTransom = 1, bit_AddDeckLid = 1, bit_RefineBowAndStern = 1,namepath = strpath)

unnormed_des = T.data_norm.transform_Data(design[0][1:])
#unnormed_des = torch.from_numpy(unnormed_des.astype('float32'))

Rt_Test = T.Predict_Drag(unnormed_des, drag_cond[0:1])

print('Predicted Drag of Test Hull: ' + str(Rt_Test[0,0]) + ' N')



45
Predicted Drag of Test Hull: 12658722.0 N
